In [5]:
## clean
!find multiplier/ ! -name 'multiplier.circom' -type f -exec rm -f {} +
!find quadratic/ ! -name 'quadratic.circom' -type f -exec rm -f {} +
!rm -rf witness.json

Introduction to ZK (Practice)

##### ZK позволяет пруверу (prover) доказать какое-то утверждение верификатору (verifier) без раскрытия части переменных. Например, доказать знание такого *m*, что *hash(m) = h*, где *h* также известно верификатору.
##### Доказательства строятся путём проведения какого-то вычисления.
##### Затем данное доказательство может быть провалидировано с помощью верификатора.
##### **Q**: Какие утверждения можно доказывать?
##### **A**: В целом можно доказывать всё что угодно, и, наверное, можно создать специализированные алгоритмы для конкретных доменов. Но самым распространённым вариантом выражения вычислений являются криптографические схемы (*Arithmetic circuits*).

![Arithmetic circuit](images/1.png)

Выше пример простой криптографической схемы. Схемы состоят из переменных и констант. Они соединяются между собой операциями сложения и умножения (в рамках поля $F$). Переменные в таких вычислениях могут быть публичными и приватными - публичные известны всем, приватные известны только пруверу и применяются при создании доказательства.

На картинке выше схема состоит из двух переменных $x_1$, $x_2$ и константы $1$. Они соединяются через сложения и умножения, и в конечном итоге получается выражение $(x_1 + x_2) \cdot (x_2 + 1) \cdot x_2$.

Это простое выражение не несет какого-то специального смысла, однако для примера можно рассмотреть его так: пусть есть верификатор, который проверяет, что пользователь знает такие $x_1$ и $x_2$, что выражение выше обращается в какое-то число $C$ (и если проверка прошла успешно, то делает что-то хорошее для прувера).

Пусть $С = 10$. Тогда одно из решений это $x_1 = 4, x_2 = 1$. Также пусть $x_1, x_2$ - приватные переменные. Для построения доказательства необходимо явно провести вычисления в этом графе, используя $x_1$ и $x_2$, и сверить после вычисления результат с константой $C$. $C$ в данном случае - публичный вход.

Верификатор получит пруф для проверки (какой-то набор чисел, сгенерированный прувером через умные zk алгоритмы), а также публичные переменные (они известны и пруверу и верификатору), и затем подаст на вход функции верификации 2 вещи: полученный пруф и публичные переменные (в данном случае $C$).

##### Выше представлена тривиальная схема. Но через такие же гейты можно выразить более сложные вещи (например, хеш-функции).

##### Представьте, что верификатор теперь проверяет не знание чисел $x_1$ и $x_2$, а знание такого сообщения $m$, что $h = sha256(m)$, где $h$ - публичная переменная. Схематично всё выглядит точно так же, как в примере выше - но применений у этого уже гораздо больше.

# Circom

##### Конечно, в реальности никто не строит такие схемы руками. Есть различные языки программирования, библиотеки и фреймворки с разными степенями абстракции для генерации доказательств и верификаторов. Сегодня познакомимся с Circom - довольно низкоуровневым фреймворком для построения криптографических схем.
#### [Документация](https://docs.circom.io/getting-started/installation/)
##### Для начала поставим нужные зависимости.

In [ ]:
!curl --proto '=https' --tlsv1.2 https://sh.rustup.rs -sSf | sh # возможно придется запустить в терминале, чтобы повыбирать пункты во время установки

In [1]:
# !git clone https://github.com/iden3/circom.git
# !cargo build --release
# !cargo install --path circom

##### Также надо поставить snarkjs, для валидации circom скриптов

In [ ]:
!npm install -g snarkjs

##### Проверям, что все работает

In [ ]:
!circom --help

##### Я буду дублировать код из файлов в ячейки для наглядности, но запускать будем файлы из multiplier/

##### Ниже представлен код из файла multiplier/multiplier.cicrom

##### Пройдемся по коду сверху вниз:
##### 1) На первой строке мы видим прагму версии компилятора - то же самое, что обычно встречается в Solidity контрактах.
##### 2) Далее идет комментарий, он выделяется символами /* ... */ или // и игнорируется при компиляции.
##### 3) Затем мы встречаем слово **template**. Темплейты позволяют выделять общие части кода в отдельную сущность. Представьте, что мы строим большую схему, и нам надо сделать много раз одно и то же умножение. Самым удобным вариантом будет вынести это в темплейт и переиспользовать по необходимости (как выглядит само переиспользование, покажу позже).
##### 4) В теле темплейта первое, что мы видим - это ключевое слово **signal**. Сигналы являются аналогами нод графа вычислений выше. После слова **signal** могут идти слова **input** или **output**. Сигналы с таким ключевым словом видны снаружи. Связывая **input** сигналы с другими сигналами, мы сможем получить **output** сигнал. Сигналы внутри темплейта без **input/output** ключевого слова не видны снаружи и могут использоваться только внутри темплейта. Последней компонентой является название сигнала, тут это *a, b, c*.
##### 5) На следующей строке мы видим констрейнт (**Constraints**). Это связи для наших сигналов. Подробнее можно найти [тут](https://docs.circom.io/circom-language/constraint-generation/). Если коротко, то такая конструкция позволяет связывать сигналы и накладывать ограничения на такие связи. Констрейнты должны быть представлены в виде A*B + C = 0. Несколько синтаксических конструкций, связанных с этим:
##### 5.1) **a * b === С** - здесь **С** - какая-то константа или другой сигнал. Такой констрейнт не соединяет сигналы, а проверяет, что равенство верно. Если при генерации доказательства такой констрейнт не удовлетворяется, то доказательство построить не получится.
###### 5.2) **C === a * b** - то же самое, что в 5.1, части равенства можно менять местами
###### 5.3) **x <-- a * b** - стрелочка служит для создания связей между сигналами, при этом данный синтаксис не генерирует и не является констрейнтом (!), а лишь присваивает сигналу x новое значение a * b. 
###### 5.4) **a * b --> x** - то же самое, что 5.3
###### 5.5) **x <== a * b** - это короткая запись двух выражений: **x <-- a * b** и **x === a * b**. Ровно такая запись встречается выше в примере. Чаще всего используется именно **<==**, а не **<--**. Позже покажу, чем **<--** может быть опасен и чем полезен. Можно писать стрелку и в другую сторону (**==>**).

##### По итогу, у нас есть темплейт Multiplier2, который принимает на вход два входа *a* и *b*, уможает их и результат умножения помещает в сигнал *c*.
##### Для генерации доказательств, нам нужно добавить точку входа для нашей схемы и скомпилировать ее. Добавим одну строку к скрипту выше:

##### Ключевое слово component позволяет инстанцировать темплейт - тут по сути мы просто говорим, что у нас будет компонент main и он будет равен Multiplier2. У main есть 2 входа *a* и *b* и на выходе мы получим выход *c*.
##### Пора компилировать! Но во что же преобразуется наш multiplier.circom? Компилятор circom генерирует *r1cs constraint system*. Это система констрейнтов (по сути система выражений вида *D = A * B + C*), которая задает ограничения для нашего утверждения. Используя код выше и опираясь на констрейнты (===, <==, ==>), он генерирует специальный файл, который позволяет создать верификатор и генерировать доказательства.
##### Итак, скомпилируем нашу схему такой командой:

In [ ]:
!circom multiplier/multiplier.circom --r1cs --wasm --sym --output multiplier

Ура!
##### На выходе мы получили несколько файлов, так как использовали несколько ключей при компиляции:
##### 1) multiplier.r1cs - основная часть, констрейнты в бинарном формате. Не важно, что там внутри, но эта вещь необходима для генерации верификатора и доказательств.
##### 2) multiplier.sym - файл для дебаггинга TODO, но из важного там видны наши сигналы - main.a, main.b и main.c
##### 3) multiplier_js/multiplier.wasm - файл, необходимый для генерации witness - об этом ниже.

Следующий шаг - генерация witness (свидетельств).
По сути это множество сигналов, используемых при генерации доказательства. Чтобы что-то доказывать, нам нужно присвоить всем сигналам конкретные значения. Однако на самом деле большинство сигналов вычисляются из других сигналов, и нам нужно задать значения только для входов - в данном случае *main.a* и *main.b*.

Создадим файл input.json и в json формате зададим значения для наших input сигналов.

In [11]:
!cd multiplier/multiplier_js && touch input.json && echo '{"a": "3", "b": "5"}' > input.json

##### Для генерации witness воспользуемся файлом ./multiplier_js/multiplier.wasm

In [12]:
!cd multiplier/multiplier_js && node generate_witness.js multiplier.wasm input.json witness.wtns

##### На выходе мы получили файлы witness.wtns и witness_calculator.js в папке multiplier_js. Посмотрим, что лежит в witness.wtns. Для этого надо превратить этот файл в читаемый json

In [ ]:
!snarkjs wtns export json multiplier/multiplier_js/witness.wtns && cat witness.json

##### И здеь мы можем увидеть значения всех посчитанных сигналов! Первый элемент всегда 1. Затем 15 - это результат, c. 3 - a, 5 - b. Итого: [1, c, a, b]

Теперь у нас есть все для создания нашего доказательства: witness.wtns и multiplier.r1cs

Но что вообще мы доказываем? В нашем примере оба входа a и b являются приватными (публичный модификатор доступа надо явно добавлять к переменным, дефолтное значение - *private*). Так как все входы у нас приватные, верификатору будет известен только выход нашего вычисления. Таким образом, мы будем доказывать, что знаем два числа *a* и *b* такие, что их произведение равно *c*.

Trusted setup

Есть много различных алгоритмов, используемых для генерации доказательств - возьмем для примера Groth16.

Первый шаг - trusted setup. Это процедура генерации определенных параметров для создания доказательств. Важно: нечестная генерация таких параметров может позволить доказывать неверные утверждения, так что эта компонента генерации доказательства очень важна.

Trusted setup состоит из двух частей:
1) Процедура powers of tau - это часть, одинаковая для всех алгоритмов
2) Phase 2 - эта часть зависит от алгоритма (у нас Groth16)

Начнем с Powers Of Tau. Инициируем церемонию:

In [15]:
import os
os.chdir("multiplier")

In [ ]:
!snarkjs powersoftau new bn128 12 pot12_0000.ptau -v

##### Теперь сделаем вклад в церемонию. В реальном случае в процедуру вносят вклад множество разных участников - это делает параметры безопасными. Никто не сможет генерировать ложные доказательства. В нашем случае мы сделаем один вклад самостоятельно.

P.S. Возможно, нужно будет ввести текст в качестве соли - это будет удобнее сделать в отдельном терминале.

In [ ]:
!snarkjs powersoftau contribute pot12_0000.ptau pot12_0001.ptau --name="First contribution" -v

##### Мы получим файл pot12_0001.ptau из файла pot12_0000.ptau
##### pot12_0001.ptau - наши параметры с первой стадии
## Phase 2

##### Начинаем генерацию этой фазы:

In [ ]:
!snarkjs powersoftau prepare phase2 pot12_0001.ptau pot12_final.ptau -v

##### И еще пара команд:

In [ ]:
!snarkjs groth16 setup multiplier.r1cs pot12_final.ptau multiplier_0000.zkey

In [ ]:
!snarkjs zkey contribute multiplier_0000.zkey multiplier_0001.zkey --name="1st Contributor Name" -v

In [ ]:
!snarkjs zkey export verificationkey multiplier_0001.zkey verification_key.json

Церемония завершена! Теперь мы можем сгенерировать доказательство.
Для этого будем использовать:
1) witness.wtns - значения для всех сигналов
2) multiplier_0001.zkey - параметры со второй фазы

In [21]:
!snarkjs groth16 prove multiplier_0001.zkey multiplier_js/witness.wtns proof.json public.json

##### На выходе мы получили 2 файла: proof.json и public.json
##### proof.json - доказательство, вернемся к нему позже.
##### public.json - все публичные сигналы нашей схемы - в данном случае это только out, результат произведения.

In [ ]:
!cat public.json

##### Теперь мы можем проверить наше доказательство (пока НЕ в виде смарт контракта):

In [ ]:
!snarkjs groth16 verify verification_key.json public.json proof.json

##### Как мы видим, никаких секретов не раскрыто!
##### Давайте сделаем магию и сгенерируем контракт верифаер:

In [ ]:
!snarkjs zkey export solidityverifier multiplier_0001.zkey MultiplyVerifier.sol

Мы получили файл MultiplyVerifier.sol!
В нем есть одна view функция - verifyProof. В качестве аргументов она принимает ранее сгенерированное доказательство и публичные инпуты (в нашем случае это результат умножения - 15), и возвращает True только если доказательство верное. Можем использовать это в наших контрактах как нам нужно!
Для генерации параметров для Solidity можно использовать эту команду:

In [ ]:
!snarkjs generatecall

### <-- vs <==
Почему важно в большинстве мест использовать <==, а не <--?
<-- не является по факту констрейнтом, а лишь соединяет сигналы между собой. Это позволяет писать более гибкие вещи, ниже пример проверки *in* на равенство нулю. Возвращаем 1, если *in == 0*, 0 иначе.
Работая с сигналом *inv*, мы используем оператор **<--**. Как мы видим, выражение справа не в форме A * B + C, а использует тернарный оператор и даже деление.

Но для генерации доказательства сам код используется только для вычисления witness значений сигналов.

Допустим, мы генерируем доказательство для входа 10.

На выходе у нас будет такой witness файл: **[1, out, in, inv] == [1, 0, 10, 1/10 % p]**

Ничего не мешает нам подменить значения в этом свидетельстве. Если какой-то сигнал не будет защищен достаточными констрейнтами, это позволит сгенерировать верное доказательство для ложного входа. Если же писать код правильно, то доказательство сгенерировать не получится.

Именно поэтому в темплейте выше есть последняя строка **in*out === 0**. Она проверит, что либо наш инпут 0, либо *out* это 0 - то есть наш *in* не 0 и мы верно посчитали его обратный элемент.

Вот пример уязвимой реализации:

##### Код выше не обладает никаким констрейтами, по сути любые значения сигналов в witness подойдут для генерации доказательств. Хотя сам код - по сути делает то, что нужно, но не влияет на доказательство.

Более сложная схема
##### Ниже пример схемы для генерации доказательства того, что пруверу известны корни некоторого квадратного уравнения.

In [ ]:
os.chdir("../quadratic")
!circom quadratic.circom --r1cs --wasm --sym

##### Пример выше принимает на вход массив сигналов *coeffs* и сигнал *x*.
##### *Coeffs* - публичные входы, их нужно будет подать верификатору на вход при верификации. *x* же является скрытым входом и не будет известен верификатору.
##### Главной точкой входа является *main*. Там мы явно указываем, что *coeffs* - публичные входы.
##### *SolveQuadraticExpression* внутри себя инициализирует несколько компонент для умножения: *x2*, *ax2* и *bx*. Входные сигналы соединяются с входами компонент через оператор *<==*, то есть запись *x2.a <== x* передает сигнал *x* на вход *x2* в качестве входа *a*. При использовании компоненты необходимо инициализировать все ее **input** входы.
##### Выходы компонент можно переиспользовать: для вычисления *a * x ^ 2* мы передаем выход компоненты *x2* на вход (так как хотим перемножить коэффициент *a* и *x^2*).
##### Для вычисления результата выражения мы используем переменную *var result*. **var** не является сигналом и служит для простой записи более сложных выражений. Поэтому после вычисления значения выражения мы явно задаем констрейнт: *result === 0*, означающий, что *x* является корнем выражения.
##### На самом деле компоненты *Multiplier2()* можно было опустить и вычислить все через var. Circom позволяет выносить сложные вычисления вне констрейнтов и верификации, но необходимо правильно сравнить результаты этих вычислений.
##### Так бы могла выглядеть упрощенная версия: